In [1]:
cd ../

/Users/zx3/PycharmProjects/scVI


In [2]:

n_epochs_all = None
save_path = 'data/'
show_plot = True


In [3]:
import numpy as np
from scvi.inference import UnsupervisedTrainer
from scvi.models.vae import VAE
from scvi.dataset import  RetinaDataset
from scvi.inference.posterior import Posterior


import torch


## Load data

In [5]:
gene_dataset = RetinaDataset(save_path=save_path)


File data/retina.loom already downloaded
Preprocessing dataset
Finished preprocessing dataset


In [12]:
nb_genes = gene_dataset.nb_genes = 13166
n_batches = gene_dataset.n_batches = 2

In [14]:
n_epochs=50 
lr=1e-3
use_batches=True

In [15]:
vae = VAE(gene_dataset.nb_genes, n_batch= n_batches * use_batches)


In [16]:
load_dict = torch.load("saved_model/ave_dict.pt",map_location='cpu')
_ = [print(key,"-", values.shape) for (key,values) in zip(load_dict.keys(),load_dict.values())]

px_r - torch.Size([13166])
z_encoder.encoder.fc_layers.Layer 0.0.weight - torch.Size([128, 13166])
z_encoder.encoder.fc_layers.Layer 0.0.bias - torch.Size([128])
z_encoder.encoder.fc_layers.Layer 0.1.weight - torch.Size([128])
z_encoder.encoder.fc_layers.Layer 0.1.bias - torch.Size([128])
z_encoder.encoder.fc_layers.Layer 0.1.running_mean - torch.Size([128])
z_encoder.encoder.fc_layers.Layer 0.1.running_var - torch.Size([128])
z_encoder.encoder.fc_layers.Layer 0.1.num_batches_tracked - torch.Size([])
z_encoder.mean_encoder.weight - torch.Size([10, 128])
z_encoder.mean_encoder.bias - torch.Size([10])
z_encoder.var_encoder.weight - torch.Size([10, 128])
z_encoder.var_encoder.bias - torch.Size([10])
l_encoder.encoder.fc_layers.Layer 0.0.weight - torch.Size([128, 13166])
l_encoder.encoder.fc_layers.Layer 0.0.bias - torch.Size([128])
l_encoder.encoder.fc_layers.Layer 0.1.weight - torch.Size([128])
l_encoder.encoder.fc_layers.Layer 0.1.bias - torch.Size([128])
l_encoder.encoder.fc_layers.Lay

In [17]:
vae.load_state_dict(load_dict)

RuntimeError: Error(s) in loading state_dict for VAE:
	size mismatch for decoder.px_decoder.fc_layers.Layer 0.0.weight: copying a param with shape torch.Size([128, 12]) from checkpoint, the shape in current model is torch.Size([128, 10]).

In [14]:
dict = vae.state_dict()
_ = [print(key,"-", values.shape) for (key,values) in zip(dict.keys(),dict.values())]

px_r - torch.Size([13166])
z_encoder.encoder.fc_layers.Layer 0.0.weight - torch.Size([128, 13166])
z_encoder.encoder.fc_layers.Layer 0.0.bias - torch.Size([128])
z_encoder.encoder.fc_layers.Layer 0.1.weight - torch.Size([128])
z_encoder.encoder.fc_layers.Layer 0.1.bias - torch.Size([128])
z_encoder.encoder.fc_layers.Layer 0.1.running_mean - torch.Size([128])
z_encoder.encoder.fc_layers.Layer 0.1.running_var - torch.Size([128])
z_encoder.encoder.fc_layers.Layer 0.1.num_batches_tracked - torch.Size([])
z_encoder.mean_encoder.weight - torch.Size([10, 128])
z_encoder.mean_encoder.bias - torch.Size([10])
z_encoder.var_encoder.weight - torch.Size([10, 128])
z_encoder.var_encoder.bias - torch.Size([10])
l_encoder.encoder.fc_layers.Layer 0.0.weight - torch.Size([128, 13166])
l_encoder.encoder.fc_layers.Layer 0.0.bias - torch.Size([128])
l_encoder.encoder.fc_layers.Layer 0.1.weight - torch.Size([128])
l_encoder.encoder.fc_layers.Layer 0.1.bias - torch.Size([128])
l_encoder.encoder.fc_layers.Lay

In [15]:
kwargs = {
    "batch_size": 128,
    "pin_memory": True
}
full = Posterior(vae, gene_dataset, indices=np.arange(len(gene_dataset)), use_cuda=False,
                          data_loader_kwargs=kwargs)


In [16]:
Post = Posterior(vae, gene_dataset, indices=np.arange(len(gene_dataset)), use_cuda=False,
                          data_loader_kwargs=kwargs)


In [19]:
_, NMI,_,_ = full.clustering_scores(prediction_algorithm = "gmm")


Clustering Scores:
Silhouette: 0.4064
NMI: 0.7976
ARI: 0.5612
UCA: 0.6158


/nfs/team205/zx3/.venv/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
/nfs/team205/zx3/.venv/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [ ]:
full.show_t_sne()
